In [1]:
#Librerias a utilizar
import requests
import re
from bs4 import BeautifulSoup
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd
from pandas.io.json import json_normalize
import json

# Web scraping

In [3]:
#creamos una lista vacia, es donde vamos a estar recibiendo los enlaces de las paginas que usaremos para el web scraping
#dentro de un while, reviso que exista la palabra https://open.spotify.com/user/
#si no aparece esa información, entonces termina el while
urls=[]
while True:
    entrada = input("Hola, introduce el link de tu enlace de perfil de spotify, puedes introducir cualquier otra cosa para salir")
    if 'https://open.spotify.com/user/' in entrada:
        urls.append(entrada)
    else:
        break

Hola, introduce el link de tu enlace de perfil de spotify, puedes introducir cualquier otra cosa para salir https://open.spotify.com/user/12101725182?si=p5aFyguOTimzgu0BvdfW3Q
Hola, introduce el link de tu enlace de perfil de spotify, puedes introducir cualquier otra cosa para salir 


In [4]:
#ejecuto el requests de cada una de las urls que ingresamos y lo guardo en una lista llamada html, 
#la cual es todo el codigo que se ejecuta en la construccion de la pagina web
#es importante aclarar que las paginas son creadas dinamicamente, por lo que este request, obtiene el codigo del script que difiere
#respecto del que en un principio se leyó
html =[requests.get(url).content for url in urls]

In [5]:
#ejecuto la expresion regular que me va a traer los usuarios de los enlaces de usuarios, esta busca el signos de ? y se trae cualquier elemento alfanumerico
#find all crea una lista, asi que por eso extraigo el elemento 0
usrs =[re.findall(r'\w*(?=\?)', url) for url in urls]
usuariosID = [elemento[0] for elemento in usrs]

In [6]:
#aqui estamos separando de todo el contenido de las html exclusivamente las playlist de la lista html
playlists = [re.findall(r'\/playlist\/\w*', str(pagina)) for pagina in html]

In [7]:
#ejecuto beautifulSoup para todas las paginas que estan en la lista html
soups = [BeautifulSoup(pagina, 'lxml') for pagina in html]

In [8]:
#busco todas las etiquetas de span, que tengan en el atributo dir el valor auto, esto nos arroja 
#una lista con los nombres de las playlist y los nombres de usuario
az = [soup.find_all('span',{'dir':'auto'}) for soup in soups]

In [9]:
#estas listas son en donde se van a guardar los nombres de las playlist y los de usuario, extrayendolos de la lista llamada az
PlstName=[]
nombres = []
for etiquetas in az:
    limpio =[]
    for contenido in etiquetas:
        #convierto a string y elimino las etiquetas de span
        limpio.append(str(contenido).replace('<span dir="auto">', '').replace('</span>', ''))
        #distingo el nombre de usuario, del nombre de la playlist
    nombres.append(limpio[0])
    PlstName.append(limpio[1:])

In [10]:
#Estas tres variables forman el nucleo final del webscraping
#en listota estoy guardando los diccionarios
listota = []
#el diccionario me separará los siguientes campos 'Id_usr', 'Nombre_perfil', 'PlaylistName','Id_playlist' que son las listas por separado
diccionario = {}
#comprimido hace tuplas de cada uno de los valores que recibe, en este caso uso listas en el orden que requiero para el diccionario
comprimido = zip(usuariosID, nombres, PlstName,playlists)

In [11]:
#Realizo la iteracion de cada elemento de comprimido (llamado rar) y despues la longitud del indice 2 (que son las playlist de los usuarios)
#y con ese campo que tiene la misma longitud de los nombres de las playlists formo el diccionario
for rar in comprimido:
    for p in range(len(rar[2])):
        diccionario= {'Id_usr':rar[0], 'Nombre_perfil':rar[1], 'PlaylistName':rar[2][p] ,'Id_playlist':rar[3][p]}
        #si no hago una copia del diccionario, siempre me va a agregar el mismo, por hacer la referencia al que se encuentra en la memoria
        listota.append(diccionario.copy())

In [12]:
#creamos el dataframe final, con los diccionarios de la listota
scrapdf = pd.DataFrame(listota)
#reinicio los indices solo para el caso de que se necesite mostrar el dataframe del webscrap
scrapdf.reset_index()

,index,Id_playlist,Id_usr,Nombre_perfil,PlaylistName
0,0,/playlist/7JBa6WHQQn7hK7Rv4JvPb4,12101725182,Erni Vil,Star Wars OST
1,1,/playlist/3LYkG7u6P8Mr5F0qwxkfPO,12101725182,Erni Vil,HBO - Tyrion Lannister GOT
2,2,/playlist/3LXuqFWKxpJGn6yoTvDAOG,12101725182,Erni Vil,Estados whats app
3,3,/playlist/37i9dQZF1E9KtIXrVxSJHB,12101725182,Erni Vil,Your Top Songs 2017
4,4,/playlist/37i9dQZF1E9WHw9mQ9boVa,12101725182,Erni Vil,Your Top Songs 2017
5,5,/playlist/37i9dQZF1E9LKo8zTDWLbT,12101725182,Erni Vil,Your Top Songs 2017
6,6,/playlist/1QpoHQzxqHrJuyGk4FOKZF,12101725182,Erni Vil,cuarteto de nos
7,7,/playlist/7mqBo8l3x7lNm7QWXwM5Bu,12101725182,Erni Vil,Mis pistas de Shazam
8,8,/playlist/37i9dQZF1DX6GJXiuZRisr,12101725182,Erni Vil,Night Rider
9,9,/playlist/37i9dQZF1DX8GjsySWIS1x,12101725182,Erni Vil,Epic Gaming


In [13]:
scrapdf.to_csv("./output/UsrPlay.csv", index = False)

# API Parsing

In [14]:
#variables para poder conectar a la API de spotifi, son provistos por developer spotify
CLIENT_ID = "2b7f57a2e7d74e43b0a0771a99fbb470"
CLIENT_SECRET = "219cfd4ed31349d5b3175e3526de4921"

#Autentificacion y uso de las variables de arriba
credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

#Creacion de los token de autenticación de spotify
token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)


In [15]:
#Creamos el diccionario base con el que comenzaremos a concatenar, esas llaves son las mismas que trae la api de spotify
apiDF = pd.DataFrame({'href':[], 'items':[], 'limit':[], 'next':[], 'offset':[], 'previous':[], 'total':[]})

In [16]:
#en esta lista, guardaremos los dataframes que resulten de la iteracion del metodo .user_playlist_tracks
dataframes = []

In [17]:
#usamos el id de usuario y el id de la playlist, que proviene del data frame del resultado anterior 
#para obtener las canciones y todos los datos que se manejan en la bd de spotify y lo guardo en una lista
for row in scrapdf.itertuples():
    canciones = spotify.user_playlist_tracks(row.Id_usr, str(row.Id_playlist).replace('/playlist/', ''))
    dataframes.append(pd.DataFrame(canciones))

In [18]:
#hacemos concat, para agregar todos los dataframes guardados en la lista a un data drafe mas grande
apiDF = pd.concat(dataframes)

In [19]:
#solo compruebo que se conservan los mismos nombres de las columnas del dataframe llamado apiDF
apiDF.columns
#reinicio los indices, para que sean continuos, si no se hiciera, no se mostraria el indice correcto
apiDF.reset_index()

,index,href,items,limit,next,offset,previous,total
0,0,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
1,1,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
2,2,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
3,3,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
4,4,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
5,5,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
6,6,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
7,7,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
8,8,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132
9,9,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,"{'added_at': '2019-02-13T12:30:48Z', 'added_by...",100,https://api.spotify.com/v1/playlists/7JBa6WHQQ...,0,None,132


In [20]:
#en candionero2, lo que hago es normalizar el json de la columna llamada items y a continuacion la imprimo solo para ver cuales son las columnas
#que se encuentran dentro de items
candionero2 = json_normalize(apiDF['items'])
print(candionero2.columns)
#creo un dataframe conformado por las columnas de candionero2, con los campos que me sirven
dfinal = candionero2[['added_by.id','track.name', 'track.external_urls.spotify', 'track.href', 'track.id'  ,'track.popularity']]

Index(['added_at', 'added_by.external_urls.spotify', 'added_by.href',
       'added_by.id', 'added_by.type', 'added_by.uri', 'is_local',
       'primary_color', 'track.album.album_type', 'track.album.artists',
       'track.album.available_markets', 'track.album.external_urls.spotify',
       'track.album.href', 'track.album.id', 'track.album.images',
       'track.album.name', 'track.album.release_date',
       'track.album.release_date_precision', 'track.album.total_tracks',
       'track.album.type', 'track.album.uri', 'track.artists',
       'track.available_markets', 'track.disc_number', 'track.duration_ms',
       'track.episode', 'track.explicit', 'track.external_ids.isrc',
       'track.external_urls.spotify', 'track.href', 'track.id',
       'track.is_local', 'track.linked_from.external_urls.spotify',
       'track.linked_from.href', 'track.linked_from.id',
       'track.linked_from.type', 'track.linked_from.uri', 'track.name',
       'track.popularity', 'track.preview_url', '

In [21]:
#guardamos el dataframe en un csv
dfinal.to_csv("./output/SpotifyApi.csv", index = False)

# limpieza de repetidos

In [22]:
dfinal.duplicated()

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
1784    False
1785    False
1786    False
1787    False
1788    False
1789    False
1790    False
1791    False
1792    False
1793    False
1794    False
1795    False
1796    False
1797    False
1798    False
1799    False
1800    False
1801    False
1802    False
1803    False
1804    False
1805     True
1806    False
1807    False
1808    False
1809    False
1810    False
1811    False
1812    False
1813    False
Length: 1814, dtype: bool

In [23]:
dduplicates = dfinal.drop_duplicates()

In [24]:
dduplicates

,added_by.id,track.name,track.external_urls.spotify,track.href,track.id,track.popularity
0,12101725182,Main Title,https://open.spotify.com/track/3ZSf1TJZyRb0rnW...,https://api.spotify.com/v1/tracks/3ZSf1TJZyRb0...,3ZSf1TJZyRb0rnWYuUtdX4,42
1,12101725182,Imperial Attack,https://open.spotify.com/track/3KVsq9veMUESIb3...,https://api.spotify.com/v1/tracks/3KVsq9veMUES...,3KVsq9veMUESIb3ln7Rowv,35
2,12101725182,Princess Leia's Theme,https://open.spotify.com/track/7hlYfQpGJWdheP2...,https://api.spotify.com/v1/tracks/7hlYfQpGJWdh...,7hlYfQpGJWdheP2quiyz8j,36
3,12101725182,The Desert and the Robot Auction,https://open.spotify.com/track/6I50fU69S2yz4yc...,https://api.spotify.com/v1/tracks/6I50fU69S2yz...,6I50fU69S2yz4yc96r55s2,32
4,12101725182,Ben's Death and TIE Fighter Attack,https://open.spotify.com/track/4bCeqjpV9iyqM3U...,https://api.spotify.com/v1/tracks/4bCeqjpV9iyq...,4bCeqjpV9iyqM3UFJuTJf3,32
5,12101725182,The Little People Work,https://open.spotify.com/track/2FMeboz4OrxtJz4...,https://api.spotify.com/v1/tracks/2FMeboz4Orxt...,2FMeboz4OrxtJz49sJsg4d,28
6,12101725182,Rescue of the Princess,https://open.spotify.com/track/15VDap1Kj0FjLe1...,https://api.spotify.com/v1/tracks/15VDap1Kj0Fj...,15VDap1Kj0FjLe1VWADhW2,28
7,12101725182,Inner City,https://open.spotify.com/track/2FNM1hkj22dtos4...,https://api.spotify.com/v1/tracks/2FNM1hkj22dt...,2FNM1hkj22dtos4SmRsjND,28
8,12101725182,Cantina Band,https://open.spotify.com/track/5ZSAdkQb23NPIcU...,https://api.spotify.com/v1/tracks/5ZSAdkQb23NP...,5ZSAdkQb23NPIcUGt6exdm,46
9,12101725182,The Land of the Sand People,https://open.spotify.com/track/21n9uPZmtIKoWnu...,https://api.spotify.com/v1/tracks/21n9uPZmtIKo...,21n9uPZmtIKoWnuzliKwgk,28


In [25]:
duplicatedsongs = dduplicates.drop_duplicates(dduplicates.columns[~dduplicates.columns.isin(['track.id', 'track.href'])], keep='first')

In [26]:
promedio = dduplicates['track.popularity'].mean()
print(promedio)
minimo = dduplicates['track.popularity'].min()
print(minimo)
maximo =  dduplicates['track.popularity'].max()
print(maximo)
Q3 = dduplicates['track.popularity'].quantile(0.75)
print(Q3)
Q1 = dduplicates['track.popularity'].quantile(0.25)
print(Q1)
IQR = dduplicates['track.popularity'].quantile(0.75)-dduplicates['track.popularity'].quantile(0.25)
print(IQR)
print(Q1 - 1.5 * IQR)
print(Q3 + 1.5 * IQR)


34.80056022408964
0
100
52.0
14.0
38.0
-43.0
109.0


In [27]:
help(spotify.current_user_top_artists)
#canciones = spotify.user_playlist_tracks(row.Id_usr, str(row.Id_playlist).replace('/playlist/', ''))
#12101725182
    

Help on method current_user_top_artists in module spotipy.client:

current_user_top_artists(limit=20, offset=0, time_range='medium_term') method of spotipy.client.Spotify instance
    Get the current user's top artists
    
    Parameters:
        - limit - the number of entities to return
        - offset - the index of the first entity to return
        - time_range - Over what time frame are the affinities computed
          Valid-values: short_term, medium_term, long_term

